In [48]:
import os

import mesa

import pandas as pd
import numpy as np

# Helper functions

In [43]:
def get_distributions(directory_path):
    # Ensure the directory path ends with a '/'
    if not directory_path.endswith('/'):
        directory_path += '/'

    try:
        all_files = os.listdir(directory_path)
    except FileNotFoundError:
        print(f"Directory {directory_path} not found.")
        return {}

    distributions = {}
    for file in all_files:
        if file.endswith('.parquet'):
            name = file.replace('.parquet', '')
            try:
                data = pd.read_parquet(os.path.join(directory_path, file))
            except Exception as e:
                print(f"Error reading {file}: {e}")
                continue

            var_type = 'cat' if ('logged_in' in name or 'service' in name or 'protocol_type' in name) else 'num'
            distributions[name] = {'data': data, 'var_type': var_type}

    return distributions

# Load distributions

## Normal

In [145]:
directory_path = f'distributions{os.sep}normal{os.sep}'
normal_agent_dist = get_distributions(directory_path)

In [146]:
directory_path = f'distributions{os.sep}smurf{os.sep}'
smurf_agent_dist = get_distributions(directory_path)

# Model

In [198]:
class NetworkModel(mesa.Model):
    def __init__(self):
        super().__init__()
        self.schedule = mesa.time.RandomActivation(self)

    def step(self):
        self.schedule.step()


# Agents

In [195]:
class KDDAgent(mesa.Agent):
    def __init__(self, unique_id, model, distributions):
        super().__init__(unique_id, model)
        self.distributions = distributions

    def get_conn_variables_from_dist(self):
        
        conn_variables = dict()
        
        for var in self.distributions:
        
            if self.distributions[var]['var_type'] == 'num':
                new_sample = np.random.choice(
                    self.distributions[var]['data']['values'], 
                    size=1, 
                    p=self.distributions[var]['data']['probs']
                )
                conn_variables[var] = new_sample[0]
        
            elif self.distributions[var]['var_type'] == 'cat':
                # Convert probabilities to cumulative probabilities
                temp_df = self.distributions[var]['data'].reset_index().copy()
                temp_df['cumulative_prob'] = temp_df['count'].cumsum()
                # Generate a random number between 0 and 1
                random_number = np.random.rand()
                # Find the index where the random number falls in the cumulative probabilities
                choice_index = temp_df['cumulative_prob'].searchsorted(random_number)
                # # Get the corresponding protocol type
                chosen_option = temp_df[var][choice_index]
            
                conn_variables[var] = chosen_option
                
        return conn_variables

class NormalAgent(KDDAgent):
    def __init__(self, unique_id, model, distributions):
        super().__init__(unique_id, model, distributions)

    def step(self):
        print('Normal')
    

class AttackerAgent(KDDAgent):
    def __init__(self, unique_id, model, distributions, attack_type):
        super().__init__(unique_id, model, distributions)
        self.attack_type = attack_type

    def step(self):
        print('Attacker')

In [199]:
model = NetworkModel()
normal_agent = NormalAgent(unique_id=1, model=model, distributions=normal_agent_dist)
smurf_agent = AttackerAgent(unique_id=1, model=model, distributions=smurf_agent_dist, attack_type='smurf')

model.schedule.add(normal_agent)
model.schedule.add(smurf_agent)

In [201]:
for i in range(1):
    model.step()

Normal
Attacker
